# Hyperparameter Optimization
This notebook will perform hyperparameter optimization for each combination of variables. Because there are many hyperparameters to run, this notebook is run in 3 different places, namely `Google Colab`, `Gradient Paperspace`, and `Personal Laptop`.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import os, json, pickle, time, math, zipfile

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

from sklearn.preprocessing import MinMaxScaler

## Define Class

In [ ]:
class TSMultistepSplit:
    """
    This class performs data splitting for Walk-Forward
    Validation with sliding window 1. The splitting data
    will be used in the LSTM for multistep forecasting or
    not (it's up to you depending on the `n_steps` parameter).
    We can divide the data into 3 categories, training, early
    stopping and testing. If you don't use early stopping,
    set `with_early_stopping_set` to `false`
    """

    def __init__(self, n_splits, n_steps, look_back):
        """
        Args:
            n_splits (int): How many fold you will used?

            n_steps (int): How many days do you predict?
            If larger than 1 its means you will perform multi-step forecasting

            look_back (int): The number of days it takes to make a prediction.
        """
        self.n_splits = n_splits
        self.n_steps = n_steps
        self.look_back = look_back

    def split(self, X, with_early_stopping_set=True):
        """
        This method returns index for training, early 
        stopping and test dataset. Below the example to
        use this class and method
        
        ```python
        
            tss = TSMultistepSplit()
            splits = tss.split(x, with_early_stopping_set=True)
            for train_indices, early_stopping_indices, test_indices in splits:
                print(train_indices, early_stopping_indices, test_indices)
                # your code        
        ```
        """
        
        n_samples = len(X)
        indices = np.arange(n_samples)

        # The number of data will used for early stopping and test data
        n_out = self.n_steps * 2 if with_early_stopping_set else self.n_steps
    
        n_train = n_samples - (n_out + self.n_splits) + 1
        if n_train < self.n_steps + self.look_back + self.n_splits:
            print("Sample size don't enough to make train data")

        for i in range(self.n_splits):
            end_train = n_train + i
            train_set = indices[:end_train]

            if with_early_stopping_set:
                early_stopping_set = indices[
                    end_train - self.look_back : 
                    end_train + self.n_steps
                ]

                test_set = indices[
                    end_train + self.n_steps - self.look_back : 
                    end_train + self.n_steps + self.n_steps
                ]

                yield train_set, early_stopping_set, test_set
            else:
                test_set = indices[
                    end_train - self.look_back : 
                    end_train + self.n_steps
                ]
                yield train_set, test_set


class DataStore():
    """
    This class is used for storing data. The format support 
    for RNN model, like LSTM.
    """
    
    def __init__(
        self, data, target_column, 
        look_back, n_steps, format=None,
        scaler_x=None, scaler_y=None, 
        default_scaler=MinMaxScaler
    ):
        """

        Args:
            data: Data will be used. `data` must have a column named `target column`
            
            target_column: Name of dependent variabel
            
            look_back: The number of days it takes to make a prediction.
            
            n_steps: How many days do you predict? If larger than 1 its means you will 
            perform multi-step forecasting
            
            format (optional): Just support RNN format. Defaults to None.
            
            scaler_x (optional): Object for transform independent data. 
            That object must have `transform` method and have been fitted. 
            If empty then the scaler to be used is the scaler defined in the 
            `default_scaler` parameter. Defaults to None.
            
            scaler_y (optional): Object for transform dependent data. 
            That object must have `transform` method and have been fitted. 
            If empty then the scaler to be used is the scaler defined in the 
            `default_scaler` parameter. Defaults to None.
            
            default_scaler (optional): The scaler must have `fit` and `transform` 
            methods. Preferably use a scaler from Scikit Learn. Defaults to MinMaxScaler.
        """
        
        self.data = data.copy()
        self.target_column = target_column
        self.look_back = look_back
        self.n_steps = n_steps
        self.n_features = data.shape[1] - 1
        
        self.scaler_x = default_scaler() if scaler_x is None else scaler_x
        self.scaler_y = default_scaler() if scaler_y is None else scaler_y
        self.defined_scaler_x = scaler_x is not None
        self.defined_scaler_y = scaler_y is not None
        
        if format == "rnn": self.format_for_rnn()
        
    def save(self, path):
        """
        To save this object to file with pickle
        """
        with open(path, 'wb') as outp:
            pickle.dump(self, outp, pickle.HIGHEST_PROTOCOL)
               
    def format_for_rnn(self):
        """
        Format data for used in RNN model that have 3 dimension, namely
        (n_data, look_back, n_features)
        """
        (x, scaled_x), (y, scaled_y) = self.__get_independent_dependent_data(self.data, self.target_column)
        
        self.x, self.y = self.__lstm_output_vector(x, y)
        self.scaled_x, self.scaled_y = self.__lstm_output_vector(scaled_x, scaled_y)
        
    def __get_independent_dependent_data(self,
                                       data : pd.DataFrame, 
                                       target_column : str) -> tuple:

        x = np.array(data)
        y = np.array(data[target_column]).reshape(-1, 1)
        
        if not self.defined_scaler_x: self.scaler_x = self.scaler_x.fit(x)
        if not self.defined_scaler_y: self.scaler_y = self.scaler_y.fit(y)
        
        scaled_x = self.scaler_x.transform(x)
        scaled_y = self.scaler_y.transform(y).squeeze()
        
        return (x, scaled_x), (y, scaled_y)
    
    def __lstm_output_vector(self, data_x: np.ndarray,  data_y: np.ndarray) -> tuple:
        x = []
        y = []
        n_data = len(data_x)
        
        for index in range(n_data):
            index_end = index + self.look_back
            index_end_output = index_end + self.n_steps
            
            if index_end_output > n_data: break
            
            x.append(data_x[index:index_end, :])
            y.append(data_y[index_end:index_end_output])

        return np.array(x), np.array(y)
    
    
class SuccessiveEarlyStopping(Callback):
    """
    This class implements the successive early stops described in the paper: 
    
    L. Prechelt, “Early Stopping - But When?,” in Neural Networks: Tricks of 
    the Trade, vol. 1524, G. B. Orr and K.-R. Müller, Eds. Berlin, Heidelberg: 
    Springer Berlin Heidelberg, 1998, pp. 55–69. doi: 10.1007/3-540-49430-8_3.
    
    This class extends Callback class from Keras Tensorflow so it's easy to
    integrate with keras model.
    """
    def __init__(self, patience=0, monitor='val_loss', min_epochs=30):
        """
        If as many as `patience` times the `monitor` goes up in a row, 
        the training process will be stopped
        """
        
        super(SuccessiveEarlyStopping, self).__init__()
        self.patience = patience
        self.monitor = monitor
        self.min_epochs = min_epochs

    def on_train_begin(self, logs=None):
        """
        This method will be executed when the modeling process starts. 
        This method will initialize some configuration
        """
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf
        self.last_epoch_loss = np.Inf
        self.best_weights_epoch = 0
        self.last_loss_down = 0

    def on_epoch_end(self, epoch, logs=None):
        """
        Every epoch end, this method will be executed to
        check if the termination criteria are met.
        """
        current = logs.get(self.monitor)
        
        if np.less(current, self.last_epoch_loss):
            if np.less(current, self.best): 
                self.best = current
                self.best_weights = self.model.get_weights()
                self.best_weights_epoch = epoch + 1
                
            self.wait = 0
            self.last_loss_down = epoch + 1
        else:
            if epoch + 1 >= self.min_epochs:
                self.wait += 1
                
            if (self.wait >= self.patience):
                self.model.stop_training = True
                print(f"Early Stopping on epoch {epoch + 1}")
                                
        self.last_epoch_loss = current
        self.stopped_epoch = epoch + 1

    def on_train_end(self, logs=None):
        """
        This method will be executed when the modeling process end.
        When the termination criteria are met. This method will 
        restore weight to best weights
        """
        if np.less(self.best, self.last_epoch_loss):
            print(f"Restoring model weights from epoch {self.best_weights_epoch} with {self.monitor} = {self.best}")
            self.model.set_weights(self.best_weights)
            

class MyTuner(kt.Tuner):
    """
    This class extends Tuner class from Keras Tuner library.
    The purpose of extending a class is to add some required methods, 
    like `get callback` and `data store`.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def __get_folder(self, trial_id, fname):
        return os.path.join(self.get_trial_dir(trial_id), fname)
        
    def save_model(self, trial_id, model, step=0):
        fname = self.__get_folder(trial_id, "model.h5")
        model.save(fname)
        
    def _save_data(self, data_store, type_data, look_back, step=0):
        fname = self.__get_folder('data', f"{type_data}_{look_back}.pkl")
        data_store.save(fname)
        
    def _save_file2json(self, trial_id, file, fname):
        fname = self.__get_folder(trial_id, fname)
        with open(fname, 'w') as f:
            f.write(json.dumps(file))
            
    def __load_model(self, trial_id):
        fname = self.__get_folder((trial_id), "model.h5")
        return tf.keras.models.load_model(fname)

    def load_model(self, trial):
        return self.__load_model(trial.trial_id)
    
    def _get_callbacks(self):
        early_stopping = SuccessiveEarlyStopping(
            patience=3, monitor='val_loss', min_epochs=40
        )
        
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=20, min_lr=1e-7
        )
        
        return early_stopping, reduce_lr
    
    def _data_store(
        self, data, index, look_back,
        scaler_x=None, scaler_y=None
    ):
        return DataStore(
            data=data.iloc[index, :],
            target_column=self.target_column,
            look_back=look_back,
            n_steps=self.n_steps,
            scaler_x=scaler_x,
            scaler_y=scaler_y,
            format="rnn"
        )
                
    def get_metrics(self, errors):
        squared_errors = np.power(errors, 2)
        abs_errors = np.abs(errors)
        return {
            'rmse_total' : np.sqrt(squared_errors.mean(axis=1)).mean(),
            'rmse_eachday' : np.sqrt(squared_errors.mean(axis=0)).tolist(),
            'mae_total' : abs_errors.mean(axis=1).mean(),
            'mae_eachday' : abs_errors.mean(axis=0).tolist()
        }
        
    def load_filejson(self, trial_id, fname):
        path_file = self.__get_folder(trial_id, fname) 
        with open(path_file, 'r') as file:
            json_file = json.load(file)
        return json_file
    
    def load_data(self, type, look_back):
        path_data = self.__get_folder('data', f"{type}_{look_back}.pkl")
        with open(path_data, 'rb') as pickle_file:
            data = pickle.load(pickle_file)
        return data
        
    def get_file_in_directory(
        self, trial_id=None
    ):
        if trial_id is None:
            trial_id = self.oracle.get_best_trials()[0].trial_id
            
        model = self.__load_model(trial_id)
        trial = self.load_filejson(trial_id, "trial.json")
        look_back = trial['hyperparameters']['values']['look_back']
        
        data_train = self.load_data("train", look_back)
        data_test = self.load_data("test", look_back) 
        data_es = self.load_data("es", look_back)           
        metrics = self.load_filejson(trial_id, "metrics.json")
        summary = self.load_filejson(trial_id, "summary_iter.json")
        
        return model, metrics, trial, summary, (data_train, data_es, data_test)
    
    
class WFVTuner(MyTuner):
    """
    This class extends MyTuner class. This class will implements 
    Walk Forward Validation with Keras Tuner. The main method 
    of this class is `run_trial` to start doing hyperparameter
    optimization
    """
    def __init__(self, target_column, n_splits, n_steps, max_epochs=100, verbose_fit_model=2, **kwargs):
        self.target_column = target_column
        self.n_splits = n_splits
        self.n_steps = n_steps
        self.max_epochs = max_epochs
        self.verbose_fit_model = verbose_fit_model
        super().__init__(**kwargs)
        
    def __get_data(self, data, train_indices, test_indices, look_back, es_indices=None):
        data_train = self._data_store(data, train_indices, look_back)
        data_test = self._data_store(
            data, test_indices, look_back,
            scaler_x=data_train.scaler_x,
            scaler_y=data_train.scaler_y
        )
        if es_indices is not None:
            data_early_stopping = self._data_store(
                data, es_indices, look_back, 
                scaler_x=data_train.scaler_x, 
                scaler_y=data_train.scaler_y
            )
            return data_train, data_early_stopping, data_test
        
        return data_train, data_test
    
    def get_prediction(self, model, x, scaler=None):
        prediction = model.predict(x)
        if scaler is not None:
            prediction = np.array(prediction).reshape(-1, 1)
            prediction = scaler.inverse_transform(prediction)
        return prediction
    
    def convert2float(self, data: list):
        return [float(x) for x in data]
    
    def run_trial(self, trial, x, y, *args, **kwargs):
        look_back = trial.hyperparameters.get('look_back')    
        batch_size = trial.hyperparameters.get('batch_size')
        
        # Split data        
        tss = TSMultistepSplit(
            n_splits=self.n_splits, n_steps=self.n_steps, look_back=look_back
        )

        errors = []
        summary_iter = []
        iteration = 1
        
        splits = tss.split(x, with_early_stopping_set=True)
        for train_indices, early_stopping_indices, test_indices in splits:
            print("Iterasi ke-", iteration)
            
            data_train, data_early_stopping, data_test = self.__get_data(
                x, train_indices, test_indices, look_back, 
                early_stopping_indices
            )
            
            early_stopping, reduce_lr = self._get_callbacks()
            
            model = self.hypermodel.build(trial.hyperparameters)
            
            start_time = time.time()
            history = model.fit(
                data_train.scaled_x, data_train.scaled_y, 
                shuffle=False, batch_size=batch_size,
                epochs=self.max_epochs, verbose=self.verbose_fit_model, 
                validation_data=(data_early_stopping.scaled_x, data_early_stopping.scaled_y), 
                callbacks=[early_stopping, reduce_lr]
            )
            training_time = time.time() - start_time
            
            # Predist early stopiing and prediction data
            prediction_es = self.get_prediction(model, data_early_stopping.scaled_x, data_early_stopping.scaler_y)
            prediction_test = self.get_prediction(model, data_test.scaled_x, data_test.scaler_y)
            error = (prediction_test - data_test.y).reshape(-1)
            errors.append(error)
            
            summary_iter.append({
                'iteration' : iteration,
                'rmse' : np.sqrt(np.power(error, 2).mean()),
                'mae' : np.abs(error).mean(),
                'best_epoch' : early_stopping.best_weights_epoch,
                'stopped_epoch' : early_stopping.stopped_epoch,
                'training_time' : training_time,
                'prediction_early_stopping' : {
                    'y_true' : data_early_stopping.y.reshape(-1).tolist(),
                    'y_pred' : prediction_es.reshape(-1).tolist()
                },
                'prediction_test' : {
                    'y_true' : data_test.y.reshape(-1).tolist(),
                    'y_pred' : prediction_test.reshape(-1).tolist(),
                },
                'history' : {
                    'loss' : self.convert2float(history.history['loss']),
                    'val_loss' : self.convert2float(history.history['val_loss']),
                    'lr' : self.convert2float(history.history['lr'])
                }
            })
            iteration += 1
            
        errors = np.array(errors)
        metrics = self.get_metrics(errors)
        
        trial_id = trial.trial_id
        self.oracle.update_trial(trial_id, {
            'val_rmse': metrics['rmse_total'],
            'val_mae': metrics['mae_total']
        })
        
        # Save model and metrics
        model._name = f"model_{trial_id}"
        self.save_model(trial_id, model)
        self._save_data(data_train, "train", look_back)
        self._save_data(data_early_stopping, 'es', look_back)
        self._save_data(data_test, "test", look_back)
        
        metrics['errors'] = errors.tolist()
        self._save_file2json(trial_id, metrics, "metrics.json")
        self._save_file2json(trial_id, summary_iter, "summary_iter.json")
        

class OneVectorHyperModel(kt.HyperModel):
    """
    This class implements HyperModel for this research
    """
    def __init__(self, n_features, n_steps, **kwargs):
        self.n_features = n_features
        self.n_steps = n_steps
        super().__init__(**kwargs)

    def build(self, hp):
        
        look_back = hp.Choice('look_back', [5, 10, 20, 30])
        batch_size = hp.Int('batch_size', 0, 64, step=16)
    
        model = Sequential()
        num_layers = hp.Int("num_layers", min_value=1, max_value=3)
        is_first_layer = True
        for i in range(num_layers):        
            
            if is_first_layer:
                model.add(Input(shape=(look_back, self.n_features)))
                is_first_layer = False
                
            is_last_layer = i == num_layers - 1
            return_sequences = not is_last_layer    
            model.add(
                LSTM(
                    units=hp.Int(f"units_{i}", min_value=100, max_value=500, step=50),
                    return_sequences=return_sequences, name=f"lstm_layer_{i}"
                )
            )
            model.add(Dropout(
                rate=hp.Float(f'dropout_rate_{i}', min_value=0, max_value=0.5), 
                name=f"dropuout_layer_{i}"
            ))

        model.add(Dense(self.n_steps, name="dense_layer_output"))
        
        optimizer = Adam if hp.Choice("optimizer", ["adam", "rmsprop"]) == "adam" else RMSprop
        learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2)
        
        model.compile(
            optimizer=optimizer(learning_rate=learning_rate), loss="mse"
        )
        
        return model

## Run Hyperparameter Optimization

In [3]:
def get_tuner(
    project_name, iterasi, n_features, n_steps=5, max_trials=30, 
    directory='model', objective='val_rmse', n_splits=10, direction='min', 
    target_column='close', verbose_fit_model=2
):
    hypermodel = OneVectorHyperModel(
        n_features=n_features,
        n_steps=n_steps
    )

    oracle = kt.oracles.BayesianOptimization(
        objective=kt.Objective(objective, direction), 
        max_trials=max_trials
    )

    return WFVTuner(
        target_column=target_column,
        n_splits=n_splits,
        n_steps=n_steps,
        oracle=oracle, 
        hypermodel=hypermodel,
        directory=directory,
        project_name=os.path.join(project_name, f'iterasi_{iterasi}'),
        verbose_fit_model=verbose_fit_model
    )

In [ ]:
bri = pd.read_csv('data/bri_data.csv').set_index('date', drop=True)

data = bri[['close']]
tuner_close = get_tuner(
    project_name='close', iterasi=4, n_splits=10,
    n_features=data.shape[1], max_trials=50, verbose_fit_model=0,
)

tuner_close.search(data, y=None)

### Get best model

In [ ]:
def get_prediction(model, data):
    prediction = model.predict(data.scaled_x)
    prediction = data.scaler_y.inverse_transform(prediction.reshape(-1, 1))
    return prediction, data.y

model, metrics, trial, summary, (data_train, data_es, data_test) = tuner_close.get_file_in_directory()
print(model.summary())

prediction1, y1 = get_prediction(model, data_es)
prediction2, y2 = get_prediction(model, data_test)
print(prediction1.reshape(-1), y1.reshape(-1))
print(prediction2.reshape(-1), y2.reshape(-1))

### Zip All Files

In [ ]:
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])


def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return convert_size(total_size)

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

print(get_size())
zipfolder('model', 'model')